In [11]:
# Built-in imports:
from typing import List
from itertools import islice

# Import GravyFlow:
import gravyflow as gf

# Dependency imports: 
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot, column

## Obtaining Transient Events

To acquire data from specific gravitational wave events (transients), use `gf.TransientObtainer`. This works similarly to `gf.NoiseObtainer` but is specifically designed for acquiring data around known event times.

### TransientObtainer

**Parameters:**

- `ifo_data_obtainer` : `gf.IFODataObtainer` (**required**):
  > The IFODataObtainer configured for transient acquisition. Unlike `NoiseObtainer`, this parameter is mandatory. The `data_labels` should include `gf.DataLabel.EVENTS` or `gf.DataLabel.GLITCHES` (not `gf.DataLabel.NOISE`).

- `ifos` : Union[`gf.IFO`, List[`gf.IFO`]] = `[gf.IFO.L1]`:
  > List of interferometers to acquire data from.

- `event_names` : Union[str, List[str]] = None:
  > Optional event name(s) to fetch (e.g., `"GW150914"` or `["GW150914", "GW170817"]`). If set, only data for these specific events will be returned, superseding the default behavior of returning all events. Event names must match those in GWTC catalogs.

- `event_types` : `List[gf.EventType]` = `[gf.EventType.CONFIDENT]`:
  > Filter by event confidence.
  > **Options:**
  > - `gf.EventType.CONFIDENT`: Confirmed detections (Default).
  > - `gf.EventType.MARGINAL`: Marginal triggers/candidates.

- `data_labels` : List[`gf.DataLabel`] = `[gf.DataLabel.EVENTS]`:
  > Specifies which transient types to include. Must NOT include `gf.DataLabel.NOISE` (raises `ValueError`). For noise acquisition, use `gf.NoiseObtainer` instead.

- `groups` : dict = `{"all": 1.0}`:
  > Group splits for data partitioning. Defaults to a single "all" group (no train/val/test split), which is typical for transient evaluation.

In [12]:
transient_obtainer = gf.TransientObtainer(
    ifo_data_obtainer=gf.IFODataObtainer(
        observing_runs=gf.ObservingRun.O3,
        data_quality=gf.DataQuality.BEST,
        data_labels=[gf.DataLabel.EVENTS],
        force_acquisition=True,               # Force the acquisition of new data.
        cache_segments=False                  # Choose not to cache the segments.
    ),
    ifos=[gf.IFO.H1, gf.IFO.L1],
    event_names=["GW150914", "GW170817"]  # Optional: specific events
)

## Searching for Events

GravyFlow provides a powerful `search_events` function to filter gravitational wave events from GWTC catalogs based on astrophysical properties, observing runs, and more.

**Parameters:**

- `source_type` : `Union[gf.SourceType, str]` = `None`:
  > Filter by astrophysical source type. 
  > **Enums (Recommended):**
  > - `gf.SourceType.BBH`: Binary Black Hole (both masses ≥ 3 M☉)
  > - `gf.SourceType.BNS`: Binary Neutron Star (both masses < 3 M☉)
  > - `gf.SourceType.NSBH`: Neutron Star - Black Hole (one < 3 M☉, one ≥ 3 M☉)
  >
  > **Strings (Supported):** `"BBH"`, `"BNS"`, `"NSBH"` (case-insensitive).

- `observing_runs` : `List[gf.ObservingRun]` = `None`:
  > Filter by specific observing runs (e.g., `[gf.ObservingRun.O3]`).

- `mass1_range` : `tuple` = `None`:
  > (min, max) range for primary mass in solar masses. Use `None` for unbounded limits. 
  > *Example:* `(30, None)` finds events with m1 > 30 M☉.

- `mass2_range` : `tuple` = `None`:
  > (min, max) range for secondary mass in solar masses.

- `total_mass_range` : `tuple` = `None`:
  > (min, max) range for total system mass (m1 + m2).

- `distance_range` : `tuple` = `None`:
  > (min, max) range for luminosity distance in Mpc.
  > *Example:* `(None, 500)` finds events closer than 500 Mpc.

- `name_contains` : `str` = `None`:
  > Substring to search for in the event name (case-insensitive).
  > *Example:* `"GW17"` matches all 2017 events.

**Returns:**
- `List[str]`: A list of event names matching all specified conditions.

In [13]:
### Examples

#### 1. Filter by Source Type (Using Enums)

# Find all Binary Neutron Star events
bns_events = gf.search_events(source_type=gf.SourceType.BNS)
print(bns_events)
# Output: ['GW170817', 'GW190425']

#### 2. Filter by Observing Run
# Find all Binary Black Holes in O3
o3_bbh = gf.search_events(
    source_type=gf.SourceType.BBH,
    observing_runs=[gf.ObservingRun.O3]
)

#### 3. Complex Physical Queries
# Find heavy BBHs (Total Mass > 80 M☉) that are relatively close (< 1000 Mpc)
heavy_nearby = gf.search_events(
    source_type=gf.SourceType.BBH,
    total_mass_range=(80, None),
    distance_range=(None, 1000)
)

#### 4. Search by Name
# Find all events from 2017
events_2017 = gf.search_events(name_contains="GW17")


2025-12-17 07:44:45,222 - INFO - Fetched 197 events with PE parameters
2025-12-17 07:44:45,229 - INFO - Fetched 100 events with PE parameters
2025-12-17 07:44:45,237 - INFO - Fetched 197 events with PE parameters
2025-12-17 07:44:45,246 - INFO - Fetched 197 events with PE parameters


['GW170817', 'GW190425']


In [14]:
print(len(gf.search_events(observing_runs=[gf.ObservingRun.O1])))
print(len(gf.search_events(observing_runs=[gf.ObservingRun.O2])))
print(len(gf.search_events(observing_runs=[gf.ObservingRun.O3])))
print(len(gf.search_events(observing_runs=[gf.ObservingRun.O4])))

2025-12-17 07:44:45,265 - INFO - Fetched 3 events with PE parameters
2025-12-17 07:44:45,273 - INFO - Fetched 8 events with PE parameters
2025-12-17 07:44:45,280 - INFO - Fetched 100 events with PE parameters
2025-12-17 07:44:45,288 - INFO - Fetched 86 events with PE parameters


3
8
100
86


In [15]:
onsource, offsource, gps_times, _ = next(gf.TransientObtainer(
    ifo_data_obtainer=gf.IFODataObtainer(
        observing_runs=gf.ObservingRun.O1,
        data_quality=gf.DataQuality.BEST,
        data_labels=[gf.DataLabel.EVENTS],
        force_acquisition=True,
        cache_segments=False
    ),
    ifos=[gf.IFO.H1, gf.IFO.L1],
    event_names=["GW150914", "GW170817"]  # Optional: specific events
)(scale_factor=1, whiten=True, crop=True))



2025-12-17 07:44:45,311 - INFO - Fetched 198 events with PE parameters
2025-12-17 07:44:45,316 - INFO - TRANSIENT MODE: 2 feature segments ready
/home/michael.norman/miniconda3/envs/gravyflow/lib/python3.13/site-packages/gwdatafind/ui.py:810: UserWarning: Missing segments: 
[1126259430.3 ... 1126259494.5000002)
  warn(msg)
/home/michael.norman/miniconda3/envs/gravyflow/lib/python3.13/site-packages/gwdatafind/ui.py:810: UserWarning: Missing segments: 
[1187008850.3 ... 1187008914.5000002)
  warn(msg)


In [16]:
gw150914_plot = gf.generate_strain_plot(
    {"Onsource Noise": onsource[0]},
    title=[
        f"L1 Onsource GW150914",
        f"H1 Onsource GW150914",
    ]
)

gw170817_plot = gf.generate_strain_plot(
    {"Onsource Noise": onsource[1]},
    title=[
        f"L1 Onsource GW170817",
        f"H1 Onsource GW170817",
    ]
)


grid = gridplot([[gw150914_plot], [gw170817_plot]])
output_notebook()
show(grid)


Loading BokehJS ...

# Glitch Acquisition

In [17]:
print("Available Glitch Types:")
glitch_types = list(gf.GlitchType)
for glitch_type in glitch_types:
    print(f"  - {glitch_type.name}: '{glitch_type.value}'")

num_glitch_types = len(glitch_types)
print(f"\nTotal: {num_glitch_types} glitch types")

Available Glitch Types:
  - AIR_COMPRESSOR: 'Air_Compressor'
  - BLIP: 'Blip'
  - CHIRP: 'Chirp'
  - EXTREMELY_LOUD: 'Extremely_Loud'
  - HELIX: 'Helix'
  - KOI_FISH: 'Koi_Fish'
  - LIGHT_MODULATION: 'Light_Modulation'
  - LOW_FREQUENCY_BURST: 'Low_Frequency_Burst'
  - LOW_FREQUENCY_LINES: 'Low_Frequency_Lines'
  - NO_GLITCH: 'No_Glitch'
  - NONE_OF_THE_ABOVE: 'None_of_the_Above'
  - PAIRED_DOVES: 'Paired_Doves'
  - POWER_LINE: 'Power_Line'
  - REPEATING_BLIPS: 'Repeating_Blips'
  - SCATTERED_LIGHT: 'Scattered_Light'
  - SCRATCHY: 'Scratchy'
  - TOMTE: 'Tomte'
  - VIOLIN_MODE: 'Violin_Mode'
  - WANDERING_LINE: 'Wandering_Line'
  - WHISTLE: 'Whistle'

Total: 20 glitch types


In [18]:

#Configure for glitch acquisition
ifo_data_obtainer = gf.IFODataObtainer(
    data_quality=gf.DataQuality.BEST,
    data_labels=[gf.DataLabel.GLITCHES],
    observing_runs=[gf.ObservingRun.O3],
    saturation=1.0,
    random_sign_reversal=False,
    random_time_reversal=False,
    balanced_glitch_types=True,
)

# Create TransientObtainer for glitches
glitch_obtainer = gf.TransientObtainer(
    ifo_data_obtainer=ifo_data_obtainer,
    ifos=[gf.IFO.L1],
)

# Create generator - get enough samples for all glitch types
# Use the new crop and whiten options for clean output
glitch_generator = glitch_obtainer(
    precache_cap=0,
    sample_rate_hertz=2048.0,
    onsource_duration_seconds=1.0,
    offsource_duration_seconds=16.0,
    num_examples_per_batch=num_glitch_types,  # One for each type
    group="all",
    scale_factor=1.0,  # No pre-scaling needed, whitening handles it
    seed=42,
    crop=True,   # Remove padding from onsource
    whiten=True  # Apply whitening (auto-scales to avoid float errors)
)

# Get batch of glitches
print("\nAcquiring glitches...")
try:
    onsource, offsource, gps_times, label = next(glitch_generator)
    print(f"Acquired {onsource.shape[0]} glitch samples")
    print(f"Onsource shape: {onsource.shape}")
    print(f"Offsource shape: {offsource.shape}")
except Exception as e:
    print(f"Error acquiring glitches: {e}")
    onsource = None

DEBUG: Non-empty indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
DEBUG: Max count: 89715
DEBUG: Balanced total times (Oversampled): 1525155


2025-12-17 07:45:10,138 - INFO - TRANSIENT MODE: 1525155 feature segments ready
2025-12-17 07:45:21,541 - INFO - Unified Data Path: cache has 109988 glitches at generator_data/glitch_cache_O3_L1.h5



Acquiring glitches...
Acquired 20 glitch samples
Onsource shape: (20, 1, 2048)
Offsource shape: (20, 1, 32768)


In [19]:
if onsource is not None:
    sample_rate = 2048.0
    
    print("\nGenerating Bokeh plots...")
    plots = []
    for i in range(min(num_glitch_types, onsource.shape[0])):        
        # Data is already whitened and cropped by TransientObtainer
        strain = onsource[i, 0, :]
        # Create strain dict for plotting function
        strain_dict = {"L1": strain}
        
        # Generate plot using gravyflow plotting function
        plot = gf.generate_strain_plot(
            strain=strain_dict,
            sample_rate_hertz=sample_rate,
            title=f"{gf.get_glitch_type_from_index(label[i])} (Whitened)",
            has_legend=False,
            height=150,
            width=800
        )
        plots.append(plot)
    
    # Combine all plots into a single column layout
    layout = column(*plots)
    
    # Save to HTML file
    output_notebook()
    show(layout)



Generating Bokeh plots...


Loading BokehJS ...

In [20]:

import numpy as np

print("Generating One-of-Each Glitch Plots...")

# Define which types to include (exclude Chirp, No_Glitch, Wandering_Line)
excluded_types = [
    gf.GlitchType.CHIRP, 
    gf.GlitchType.NO_GLITCH,
    gf.GlitchType.WANDERING_LINE,  # Currently broken in Gravity Spy
]
target_types = [gt for gt in gf.GlitchType if gt not in excluded_types]

plots = []

# Iterate through each type and create a dedicated obtainer
for glitch_type in target_types:
    print(f"\nProcessing {glitch_type.name}...")
    
    try:
        # Pass the specific GlitchType directly in data_labels!
        # This is the clean API - no hacks needed.
        ifo_data_obtainer = gf.IFODataObtainer(
            data_quality=gf.DataQuality.BEST,
            data_labels=[glitch_type],  # <-- Pass GlitchType directly here
            observing_runs=[gf.ObservingRun.O3],
            saturation=1.0,
            random_sign_reversal=False,
            random_time_reversal=False,
        )
        
        glitch_obtainer = gf.TransientObtainer(
            ifo_data_obtainer=ifo_data_obtainer,
            ifos=[gf.IFO.L1],
        )
        
        # Generate ONE sample of this type
        glitch_generator = glitch_obtainer(
            precache_cap=0,  # Lazy download, no caching
            sample_rate_hertz=2048.0,
            onsource_duration_seconds=1.0,
            offsource_duration_seconds=16.0,
            num_examples_per_batch=1, 
            group="all",
            crop=True,
            whiten=True
        )
        
        onsource, offsource, gps_times, labels = next(glitch_generator)
        
        if onsource is None or onsource.shape[0] == 0:
            print("  - No data returned.")
            continue
        
        # Check for NaNs
        if np.isnan(onsource).any():
            print(f"  - WARNING: NaN detected in {glitch_type.name}!")
            continue

        strain = onsource[0, 0, :]

        print(strain.shape)
        
        # Plot
        gps_val = np.asarray(gps_times[0]).item()
        plot = gf.generate_strain_plot(
            strain={"L1": np.asarray(strain)},
            sample_rate_hertz=2048.0,
            title=f"{glitch_type.name} (GPS: {gps_val:.1f})",
            has_legend=False,
            height=150,
            width=800
        )
        plots.append(plot)
        print(f"  - Success: {glitch_type.name} plotted.")
        
    except Exception as e:
        print(f"  - Error processing {glitch_type.name}: {e}")

# Combine
if plots:
    print(f"\nSuccessfully generated {len(plots)} plots.")
    layout = column(*plots)

    # Save to HTML file
    output_notebook()
    show(layout)
else:
    print("No plots generated.")



2025-12-17 07:45:59,921 - INFO - TRANSIENT MODE: 6609 feature segments ready
2025-12-17 07:46:00,027 - INFO - Unified Data Path: cache has 110008 glitches at generator_data/glitch_cache_O3_L1.h5


Generating One-of-Each Glitch Plots...

Processing AIR_COMPRESSOR...
(2048,)
  - Success: AIR_COMPRESSOR plotted.

Processing BLIP...


2025-12-17 07:46:01,173 - INFO - TRANSIENT MODE: 25013 feature segments ready
2025-12-17 07:46:01,410 - INFO - Unified Data Path: cache has 110008 glitches at generator_data/glitch_cache_O3_L1.h5


(2048,)
  - Success: BLIP plotted.

Processing EXTREMELY_LOUD...


2025-12-17 07:46:02,391 - INFO - TRANSIENT MODE: 7291 feature segments ready
2025-12-17 07:46:02,471 - INFO - Unified Data Path: cache has 110008 glitches at generator_data/glitch_cache_O3_L1.h5
2025-12-17 07:46:03,269 - INFO - TRANSIENT MODE: 758 feature segments ready
2025-12-17 07:46:03,293 - INFO - Unified Data Path: cache has 110008 glitches at generator_data/glitch_cache_O3_L1.h5


...

2025-12-17 07:46:15,776 - INFO - TRANSIENT MODE: 28412 feature segments ready
2025-12-17 07:46:16,036 - INFO - Unified Data Path: cache has 110008 glitches at generator_data/glitch_cache_O3_L1.h5
2025-12-17 07:46:16,904 - INFO - TRANSIENT MODE: 2171 feature segments ready
2025-12-17 07:46:16,981 - INFO - Unified Data Path: cache has 110008 glitches at generator_data/glitch_cache_O3_L1.h5


(2048,)
  - Success: TOMTE plotted.

Processing VIOLIN_MODE...
(2048,)
  - Success: VIOLIN_MODE plotted.

Processing WHISTLE...


2025-12-17 07:46:17,840 - INFO - TRANSIENT MODE: 6596 feature segments ready
2025-12-17 07:46:17,874 - INFO - Unified Data Path: cache has 110008 glitches at generator_data/glitch_cache_O3_L1.h5


(2048,)
  - Success: WHISTLE plotted.

Successfully generated 16 plots.


Loading BokehJS ...